In [1]:
import verde as vd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import pyproj
from shapely import geometry


%matplotlib widget

gdb = '/home/ggrl/geodatabase/'

In [2]:
g_rparaim = pd.read_csv(gdb+'geof/recortado/g_rparaim')

In [3]:
g_rparaim.sort_values('LATITUDE')

,X,Y,MDT,KPERC,eU,eth,CTCOR,LONGITUDE,LATITUDE
37611,523111.29,8784013.32,484.62,0.37,1.27,7.40,695.04,-44.788444,-10.999992
51744,533602.25,8784004.71,452.52,0.48,0.47,7.41,625.33,-44.692412,-10.999988
65711,543613.15,8783993.73,452.72,0.22,0.41,5.58,436.27,-44.600776,-10.999980
62911,541130.10,8783998.16,443.86,0.15,0.49,3.56,310.58,-44.623504,-10.999969
32024,519113.19,8784018.60,516.49,0.85,1.52,4.85,726.11,-44.825041,-10.999967
...,...,...,...,...,...,...,...,...,...
17444,509093.55,8839297.36,426.09,0.12,1.28,8.20,668.69,-44.916896,-10.500057
78993,552606.04,8839259.22,490.75,0.60,1.35,15.88,1256.65,-44.519250,-10.500049
58049,537617.81,8839279.49,437.43,0.46,0.87,8.99,759.14,-44.656221,-10.500044
43343,527623.79,8839292.72,441.65,0.27,1.92,14.35,1145.37,-44.747553,-10.500010


In [4]:
coordinates = (g_rparaim.X.values, g_rparaim.Y.values)
coordinates

(array([500020.25, 500103.83, 500187.24, ..., 554618.94, 554618.52,
        554617.97]),
 array([8786155.43, 8786158.77, 8786161.64, ..., 8839028.43, 8839107.8 ,
        8839187.24]))

In [12]:
plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c    = g_rparaim.CTCOR,
            cmap = 'hsv',
            s    = 2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
g_rparaim['geometry'] = [geometry.Point(x,y) for x, y in zip(g_rparaim['LONGITUDE'], g_rparaim['LATITUDE'])]

In [7]:
# Block Reduction
reducer_median = vd.BlockReduce(np.mean, spacing= 500)
b_coords, b_CTCOR = reducer_median.filter(coordinates, g_rparaim.CTCOR)


# Ploting the Reduced data
plt.figure('Spacing 500m',figsize=(11,11))
plt.scatter(b_coords[0],b_coords[1],
            c    = b_CTCOR,
            cmap = 'magma',
            s    = 2)
plt.axis('scaled')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
reducer_mean = vd.BlockReduce(np.mean, spacing= 499)
b_coords, b_CTCOR = reducer_mean.filter(coordinates, g_rparaim.CTCOR)

plt.figure('Spacing 499',figsize=(16,16))
plt.scatter(b_coords[0],b_coords[1],
            c    = b_CTCOR,
            cmap = 'hsv',
            s    = 2)
plt.axis('scaled')
plt.colorbar()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
spline = vd.Spline()

In [26]:
spline.fit(b_coords, b_CTCOR)

In [ ]:
predicted = spline.predict(coordinates)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c=predicted,
            cmap='magma',
            s=1)
plt.axis('scaled')
plt.colorbar()

In [ ]:
residuals = g_rparaim.CTCOR - predicted

In [ ]:
scale = vd.maxabs(residuals)

plt.figure(figsize=(8,8))
plt.scatter(coordinates[0],coordinates[1],
            c=residuals,
            cmap='RdBu_r',
            s=2,
            vmin=-scale,vmax=scale)
plt.axis('scaled')
plt.colorbar()

In [ ]:
region = vd.get_region(coordinates)
grid_coords = vd.grid_coordinates(region, spacing = 200,pixel_register=True)

In [ ]:
region

In [ ]:
grid_CTCOR = spline.predict(grid_coords)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(grid_coords[0], grid_coords[1],
            c=grid_CTCOR,
            cmap='magma',
            s=2)
plt.axis('scaled')
plt.colorbar()

In [ ]:
grid = spline.grid(spacing=200, data_names=['CTCOR'])
grid

In [ ]:
'''
vmin= grid.CTCOR.quantile([0.25,0.5,0.75])
print(vmin)
'''
grid.CTCOR.plot(figsize=(12,12),
               cmap='hsv',vmin=600, vmax=2000)
plt.axis('scaled')

In [ ]:
grid = vd.distance_mask(coordinates, maxdist=1000, grid=grid)
grid

grid.CTCOR.plot(figsize=(12,12),cmap='hsv')
plt.axis('scaled')

In [ ]:
chain = vd.Chain([
    ('trend',  vd.Trend(degree=1)),
    ('reduce', vd.BlockReduce(np.median, spacing=2000)),
    ('spline', vd.Spline())
])

In [ ]:
g_rparaim.head()

In [ ]:
chain.fit(coordinates, g_rparaim.KPERC)

grid = chain.grid(spacing=400, data_names=['KPERC'])
grid_mask = vd.distance_mask(coordinates, maxdist=2000, grid=grid)

In [ ]:
grid_mask

In [ ]:
grid_mask.KPERC.plot(figsize=(8,8), cmap='hsv',vmin=0)
plt.axis('scaled')

Model Validation

In [ ]:
train, test = vd.train_test_split(coordinates, g_rparaim.CTCOR,
                                 test_size = 0.1)

In [ ]:
train

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(train[0][0], train[0][1], '.b', markersize=4)
plt.plot(test[0][0], test[0][1], '.r', markersize=4)
plt.axis('scaled')

In [ ]:
chain_CTCOR.fit(*train)
# *train   ---> chain.fit(train[0], train[1])

In [ ]:
chain_CTCOR.score(*test)

In [ ]:
train, test = vd.train_test_split(coordinates, g_rparaim.CTCOR, test_size=0.1, spacing=1000)

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(train[0][0], train[0][1], '.b', markersize=3)
plt.plot(test[0][0], test[0][1], '.r', markersize=3)
plt.axis('scaled')

In [ ]:
chain_CTCOR.fit(*train)

In [ ]:
chain_CTCOR.score(*test)

In [ ]:
cv     = vd.BlockKFold(spacing=1000,
                  n_splits=50,
                  shuffle=True)

In [ ]:
scores = vd.cross_val_score(chain_CTCOR,
                            coordinates,
                            g_rparaim.CTCOR,
                            cv=cv)

In [ ]:
scores

In [ ]:
plt.figure()
plt.hist(scores, bins ='auto')

In [ ]:
geof = CTCOR_grid.to_dataframe()
geof.reset_index(inplace=True)

In [ ]:
geof